# The idea

I have an idea for my variability spreadsheet code (an idea that may be more eloquent and panda-onic (like pythonic?) than my NamedTuples approach): using the approach advocated [here](https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns/47103408).

I'm curious whether I can make it work.

In [3]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.rand(4,4), columns=list('abcd'))
df['group'] = [0, 0, 1, 1]
df

,a,b,c,d,group
0,0.148689,0.027214,0.167169,0.934639,0
1,0.335818,0.488526,0.942086,0.168513,0
2,0.846428,0.546210,0.234183,0.585473,1
3,0.342202,0.735204,0.237963,0.734686,1


In [4]:
df.groupby('group').agg({'a':['sum', 'max'], 
                         'b':'mean', 
                         'c':'sum', 
                         'd': lambda x: x.max() - x.min()})

a                   b         c         d
            sum       max      mean       sum  <lambda>
group                                                  
0      0.484507  0.335818  0.257870  1.109254  0.766126
1      1.188630  0.846428  0.640707  0.472146  0.149213

In [7]:
def max_min(x):
    return x.max() - x.min()

max_min.__name__ = 'Max minus Min'

df.groupby('group').agg({'a':['sum', 'max'], 
                         'b':'mean', 
                         'c':'sum', 
                         'd': max_min})

a                   b         c             d
            sum       max      mean       sum Max minus Min
group                                                      
0      0.484507  0.335818  0.257870  1.109254      0.766126
1      1.188630  0.846428  0.640707  0.472146      0.149213

In [8]:
def f(x):
    d = {}
    d['a_sum'] = x['a'].sum()
    d['a_max'] = x['a'].max()
    d['b_mean'] = x['b'].mean()
    d['c_d_prodsum'] = (x['c'] * x['d']).sum()
    return pd.Series(d, index=['a_sum', 'a_max', 'b_mean', 'c_d_prodsum'])

df.groupby('group').apply(f)

,a_sum,a_max,b_mean,c_d_prodsum
group,,,,
0,0.484507,0.335818,0.257870,0.314996
1,1.188630,0.846428,0.640707,0.311936


In [12]:
def f_mi(x):
    d = []
    d.append(x['a'].sum())
    d.append(x['a'].max())
    d.append(x['b'].mean())
    d.append((x['c'] * x['d']).sum())
    return pd.Series(d, index=[['a', 'a', 'b', 'c_d'], 
                               ['sum', 'max', 'mean', 'prodsum']])

multi_groupby = df.groupby('group').apply(f_mi)
multi_groupby

a                   b       c_d
            sum       max      mean   prodsum
group                                        
0      0.484507  0.335818  0.257870  0.314996
1      1.188630  0.846428  0.640707  0.311936

In [17]:
multi_groupby['a']['max']

group
0    0.335818
1    0.846428
Name: max, dtype: float64

okay, so what if we had like... 3 types of variables (J, H, K), and we wanted to do mean, median, min, max, range, ... let's try it with mean/median/std for J and H following the syntax they've got: